# Annotate merged single cells with metadata from platemap file for each plate

## Import libraries

In [1]:
import sys
import pathlib
import yaml
import pprint

import pandas as pd
from pycytominer import annotate
from pycytominer.cyto_utils import output

sys.path.append("../utils")
import extraction_utils as sc_utils

## Set paths and variables

In [2]:
# output directory for annotated data
output_dir = pathlib.Path("./data/annotated_data")
output_dir.mkdir(exist_ok=True)

# directory with metadata
metadata_dir = pathlib.Path("../0.download_data/metadata/")

# load in dicionary from yaml file
dictionary_path = pathlib.Path("./plate_info_dictionary.yaml")
with open(dictionary_path) as file:
    plate_info_dictionary = yaml.load(file, Loader=yaml.FullLoader)

## Add metadata paths to loaded in dictionary

In [3]:
# add path to platemaps for each plate 
for plate in plate_info_dictionary.keys():
    # since Plate_3_prime has the same platemap as Plate_3, we need an else statement so that we make sure it adds the 
    # path that was given to Plate_3
    if plate != "Plate_3_prime":
        # match the naming format of the plates to the platemap file
        plate_info_dictionary[plate]["platemap_path"] = str(
            pathlib.Path(list(metadata_dir.rglob(f"platemap_NF1_{plate.replace('_', '').lower()}.csv"))[0]).resolve(
                strict=True
            )
        )
    else:
        plate_info_dictionary["Plate_3_prime"]["platemap_path"] = plate_info_dictionary["Plate_3"]["platemap_path"]

# view the dictionary to assess that all info is added correctly
pprint.pprint(plate_info_dictionary, indent=4)

{   'Plate_1': {   'dest_path': 'data/converted_data/Plate_1.parquet',
                   'platemap_path': '/home/jenna/nf1_cellpainting_data/0.download_data/metadata/platemap_NF1_plate1.csv',
                   'source_path': '/home/jenna/nf1_cellpainting_data/2.cellprofiler_analysis/analysis_output/Plate_1/Plate_1_nf1_analysis.sqlite'},
    'Plate_2': {   'dest_path': 'data/converted_data/Plate_2.parquet',
                   'platemap_path': '/home/jenna/nf1_cellpainting_data/0.download_data/metadata/platemap_NF1_plate2.csv',
                   'source_path': '/home/jenna/nf1_cellpainting_data/2.cellprofiler_analysis/analysis_output/Plate_2/Plate_2_nf1_analysis.sqlite'},
    'Plate_3': {   'dest_path': 'data/converted_data/Plate_3.parquet',
                   'platemap_path': '/home/jenna/nf1_cellpainting_data/0.download_data/metadata/platemap_NF1_plate3.csv',
                   'source_path': '/home/jenna/nf1_cellpainting_data/2.cellprofiler_analysis/analysis_output/Plate_3/Plate_3_

## Annotate merged single cells

**Note:** The path to the annotated file to be used for normalization is adding during this step.

In [4]:
for plate, info in plate_info_dictionary.items():
    # single_cell_df is the dataframe loaded in from the converted parquet file
    single_cell_df = pd.read_parquet(info["dest_path"])
    platemap_df = pd.read_csv(info["platemap_path"])
    output_file = str(pathlib.Path(f"{output_dir}/{plate}_sc.parquet"))
    # save path to annotated file to dictionary for downstream use
    plate_info_dictionary[plate]["annotated_path"] = output_file
    print(f"Adding annotations to merged single cells for {plate}!")

    # add metadata from platemap file to extracted single cell features
    annotated_df = annotate(
        profiles=single_cell_df,
        platemap=platemap_df,
        join_on=["Metadata_well_position", "Image_Metadata_Well"],
    )

    # rename site column to avoid any issues with identifying the column as metadata over feature
    annotated_df = annotated_df.rename(columns={"Image_Metadata_Site": "Metadata_Site"})

    # move metadata well, single cell count, and site to the front of the df (for easy visualization in python)
    well_column = annotated_df.pop("Metadata_Well")
    singlecell_column = annotated_df.pop("Metadata_number_of_singlecells")
    site_column = annotated_df.pop("Metadata_Site")    

    # insert the columns in specific parts of the dataframe
    annotated_df.insert(2, "Metadata_Well", well_column)
    annotated_df.insert(3, "Metadata_Site", site_column)
    annotated_df.insert(4, "Metadata_number_of_singlecells", singlecell_column)

    # save annotated df as parquet file
    output(
        df=annotated_df,
        output_filename=output_file,
        output_type="parquet",
    )
    print(f"Annotations have been added to {plate} and saved!")

Adding annotations to merged single cells for Plate_1!
Annotations have been added to Plate_1 and saved!
Adding annotations to merged single cells for Plate_2!
Annotations have been added to Plate_2 and saved!
Adding annotations to merged single cells for Plate_3!
Annotations have been added to Plate_3 and saved!
Adding annotations to merged single cells for Plate_3_prime!
Annotations have been added to Plate_3_prime and saved!
Adding annotations to merged single cells for Plate_4!
Annotations have been added to Plate_4 and saved!


In [5]:
# print last annotated df to see if annotation occurred
print(annotated_df.shape)
annotated_df.head()

(7502, 1600)


,Metadata_WellRow,Metadata_WellCol,Metadata_Well,Metadata_Site,Metadata_number_of_singlecells,Metadata_gene_name,Metadata_genotype,Metadata_seed_density,Metadata_siRNA,Metadata_RNAiMax,...,Nuclei_Texture_Variance_DAPI_3_02_256,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GFP_3_00_256,Nuclei_Texture_Variance_GFP_3_01_256,Nuclei_Texture_Variance_GFP_3_02_256,Nuclei_Texture_Variance_GFP_3_03_256,Nuclei_Texture_Variance_RFP_3_00_256,Nuclei_Texture_Variance_RFP_3_01_256,Nuclei_Texture_Variance_RFP_3_02_256,Nuclei_Texture_Variance_RFP_3_03_256
0,B,2,B2,11,115,NF1,WT,1000,NaN,0,...,886.555259,887.126996,220.059165,178.494944,177.175567,178.536364,1947.689117,1942.505237,1929.883349,1954.302782
1,B,2,B2,11,115,NF1,WT,1000,NaN,0,...,2269.739602,2183.665041,197.803849,181.889009,190.645422,197.228087,224.801646,219.118772,235.706020,228.832375
2,B,2,B2,11,115,NF1,WT,1000,NaN,0,...,993.492352,951.907468,424.891364,390.009779,381.521734,390.667328,623.836472,616.211140,661.626364,636.520133
3,B,2,B2,11,115,NF1,WT,1000,NaN,0,...,2636.424708,2517.017120,561.624256,544.408328,548.657987,556.701996,364.792509,365.727202,437.796494,380.910214
4,B,2,B2,14,115,NF1,WT,1000,NaN,0,...,652.339728,635.869580,126.833034,120.283432,109.813358,109.301294,299.201582,297.223402,320.864494,303.344477


## Write updated dictionary to yaml file for use in downstream steps

In [6]:
with open(dictionary_path, "w") as file:
    yaml.dump(plate_info_dictionary, file)